In [1]:
import os 
import pandas as pd 
import glob
import numpy as np 
from functools import reduce
import operator

In [2]:
datafromR_raw = {
    'morph': '''2      mod 222.3750 1.0878113 163.40278 0.6191000
3     high 124.3125 2.4689741  90.64000 0.4131859
4  modhigh 173.9375 1.5692355 129.14035 0.5784197
5 veryhigh  85.6250 2.1563859  60.74194 0.5878860''',
    'nomorph': '''2      mod 202.3750 1.0878113 148.80000 0.7462130
3     high 104.3125 2.4689741  78.48485 0.4194891
4  modhigh 153.9375 1.5692355 115.02500 0.7341812
5 veryhigh  65.6250 2.1563859  45.35714 1.1705462'''
} 

In [3]:
# data = {
# }

# for m, rawtext in datafromR_raw.iteritems():
#     data_ = {}
#     for row in rawtext.split('\n'):
#         items = row.split()
#         config = items[1]
#         a = items[2]
#         b = items[4]
#         data_[config] = [0] * 6
#         data[config]
#     data[m] = data_


In [4]:
# FACTORS = ["0.5", "1.0", "2.0", "4.0", "8.0"]
# POLYMORPHISM = ["low", "mod", "modhigh", "high", "veryhigh"]
FACTORS = [ "1.0", "2.0", "4.0", "8.0"]
POLYMORPHISM = ["mod", "modhigh", "high", "veryhigh"]

In [5]:
def count_states(values):
    # a list of lists of states that each taxa exhibits
    all_chars = [(a.split('/')) for a in values]
    # e.g. [['0', '1'], ['0'], ['2']]
    # flattens the list, remove duplicates, and take the length
    return len(set(reduce(
        operator.add,
        all_chars,
        []
    )))

In [6]:
def get_setting(f, p, morph_chars=False):
    if morph_chars:
        csvs = glob.glob(f"./example/simulated_data-{f}/{p}_noborrowing/*.csv")
    else:
        csvs = glob.glob(f"./example/simulated_data-{f}/{p}_noborrowing/no-morph/*.csv")

    TREES = 8
    REPLICAS = 2
    goodcsvs = [f"sim_tree{t}_{r}.csv" for t in range(1, TREES+1) for r in range(1, 1+REPLICAS)]
    csvs = list(filter(
        lambda c: any([mask in c for mask in goodcsvs]),
        csvs
    ))
    
    means = []
    for csv_path in csvs:
        df = pd.read_csv(csv_path).iloc[:, 3:].astype(str) # remove metadata and cast to string
        all_values = np.array(list(map(count_states, df.values)))
        means.append(all_values.mean()) # get mean no. of states per character for this dataset
    
    # print(f'{p}{" " * (10 - len(p))}\t${np.array(means).mean():.2f}$') # print overall mean for this setting
    return np.array(means).mean()

In [7]:

for m in [False, True]:
    print(f"====== m = {m} ====== ")
    for p in POLYMORPHISM:
        num_chars = []
        for f in ["1.0", "8.0"]:
            # print(f'f = {f}')
            num_chars.append(get_setting(f, p, morph_chars=m))
        print(f'{p} &  & & ${num_chars[0]:.2f}$ & & & ${num_chars[1]:.2f}$ \\\\')

====== m = False ====== 
mod &  & & $14.88$ & & & $27.55$ \\
modhigh &  & & $15.42$ & & & $28.27$ \\
high &  & & $15.76$ & & & $28.74$ \\
veryhigh &  & & $15.95$ & & & $29.14$ \\
====== m = True ====== 
mod &  & & $14.52$ & & & $27.29$ \\
modhigh &  & & $15.03$ & & & $27.96$ \\
high &  & & $15.35$ & & & $28.41$ \\
veryhigh &  & & $15.53$ & & & $28.79$ \\


In [8]:
import re 

In [9]:
t = '''Low &  &$134.96$ &$5.24$& $265.18$ & $120.05$&$9.10$\\
Moderate &  & $116.46$ &$5.41$& $227.61$ & $103.58$&$9.32$\\
Mod. High & & $98.67$ &$5.55$& $192.5625$ & $88.60$&$9.51$\\
High &  & $82.25$ &$5.70$& $155.6406$ & $72.85$&$9.73$\\
Very High &  &  &&  & $59.33$&$9.86$\\'''

In [10]:
df = pd.read_csv("/home/zxliu2/QuartetMethods/example/simulated_data_small-8.0/veryhigh_noborrowing/sim_tree1_1.csv")

In [11]:
row = df.values[100]
print(row)

['lpmh0' 'lpmh0' 1.0 '2' '2' '84' '82' '74/75' '61' '0/73' '0/335' '78'
 '80/75']


In [12]:
class Frequenter():
    # supports initialisation with a list of items
    # keeps a frequency table 
    # supports querying of a list of items, returning the most frquent out of them. 
    # ties are broken arbitrarily.
    def __init__(self, arr):
        self.frqs = {}
        for x in arr:
            if x not in self.frqs:
                self.frqs[x] = 0
            self.frqs[x] += 1
    
    def query(self, k):
        return max(k, key=lambda x : self.frqs[x])



In [13]:
def process_row(row):
    data = row[3:]
    data = [x.split('/') for x in data]
    frq = Frequenter(reduce(
        lambda acc, x : acc + x,
        data,
        [] 
    ))
    data = [ frq.query(k) for k in data ]
    row[3:] = data
    return row

In [14]:
df.values[0][2] = 7122

In [15]:
for i, row in enumerate(df.values):
    df.iloc[i, :] = process_row(row)

In [4]:
import glob, shutil, os
import tqdm

In [5]:
to_del = glob.glob("/home/zxliu2/QuartetMethods/outputs_small/*/*/*/ASTRAL([0-9],[0-9])")

In [7]:
for file in tqdm.tqdm(to_del):
    shutil.rmtree(file)

0it [00:00, ?it/s]
